# Clasificación de noticias mediante embedding (zero shot learning)


In [ ]:
# Comenzamos instalando las dependencias necesarias
!pip install transformers # Librería para interactuar con los modelos de Hugging Face
!pip install requests # Librería para hacer requests a sitios webs
!pip install beautifulsoup4 # Librería para extraer contenido de sitios webs

In [ ]:
#Importamos las librerías que vamos a utilizar
import requests
from bs4 import BeautifulSoup
from transformers import pipeline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Ahora procedemos a seleccionar el sitio web de noticias que analizaremos






In [ ]:
url = "https://www.pagina12.com.ar"
response = requests.get(url) # Request al sitio web
soup = BeautifulSoup(response.text, 'html.parser') # Extrae el texto

In [ ]:
soup.prettify() # Veamos el contenido de esta cosa


'<!DOCTYPE html>\n<!-- Version 2.0.284 -->\n<html class="no-js" lang="es">\n <head>\n  <meta charset="utf-8"/>\n  <title>\n   Página | 12: La otra mirada sobre Argentina y el mundo\n  </title>\n  <meta content="x6zSdT0DBcKDmridH4LpEVrCmxcOunR2dgBQVmuL6fg" name="google-site-verification"/>\n  <link href="https://www.pagina12.com.ar" rel="canonical"/>\n  <link href="https://cdn.jsdelivr.net/npm/glider-js@1/glider.min.css" rel="stylesheet"/>\n  <script src="https://cdn.jsdelivr.net/npm/glider-js@1/glider.min.js">\n  </script>\n  <script type="application/ld+json">\n   {"@context": "http://schema.org","@type": "Organization","name": "Página|12","url": "https://www.pagina12.com.ar","logo": {"@type": "ImageObject","caption": "Logo de Página|12","url": "https://www.pagina12.com.ar/assets/media/logo_default_p12.png","width": "600","height": "60"},"sameAs":["https://twitter.com/pagina12","https://www.youtube.com/channel/UCJNDedOnljCssaiRZqg8-Dg","https://www.instagram.com/pagina12/","https://ww

In [ ]:
print(soup.get_text()) # Ahora extraigamos el texto


Página | 12: La otra mirada sobre Argentina y el mundo  22 de febrero de 2025Edición Impresa  PDFNewsletterAsociateIniciar sesiónRelatores - EN VIVOAsociateIniciar sesiónEdición Impresa  PDF22 de febrero de 2025Hoy:Verano12Buenos Aires12NOSoyLas12Secciones y SuplementosHoy:Verano12Buenos Aires12NOSoyLas12SeccionesEl paísEconomíaSociedadEspectáculosDeportesEl mundoCienciaCulturaBuenos Aires12Rosario12Salta12NegrxUniversidadAudiovisualesPlásticaPsicologíaSoci@sLa ventanaContratapaDiálogosRecordatoriosEdición impresaSuplementosCultura y EspectáculosCashRadarRadar LibrosNOSoyLas12Verano12Especiales de P12MediosGO NoticiasAM750MalenaLatinoamérica PiensaEl paísEconomíaSociedadEspectáculosDeportesEl mundoCienciaHoy:Verano12Buenos Aires12NOSoyLas12Página 12 Hayden Davis será denunciante y querellante en la causa del criptogateEntra a escena el hombre que compromete a MileiLa abogada del joven señalado como uno de los principales impulsores de $Libra informó que se pusieron a disposición de la 

In [ ]:
# Vamos a ver todos los links a noticias que hay en la portada
links =[]
for link in soup.find_all('a'):
    links.append(link.get('href'))
links = list( set(links))
print( links[0:20])

['https://fmblackie.com.ar/', 'https://www.pagina12.com.ar/805879-clima-en-buenos-aires-el-pronostico-del-tiempo-para-este-sab', 'https://www.pagina12.com.ar/humor/rep/805873', 'https://www.pagina12.com.ar/am750/radio-en-vivo', 'https://fmaspen.com/', 'https://www.pagina12.com.ar/805790-un-legislador-de-lla-contra-karina-tengo-dudas-de-su-transpa', 'https://www.pagina12.com.ar/805829-brasil-la-justicia-desactivo-la-cuenta-de-x-de-jair-bolsonar', 'https://www.pagina12.com.ar/805964-petro-contra-la-motosierra-de-milei-en-colombia-se-uso-para-', None, 'https://www.pagina12.com.ar/autores/297909-euge-murillo', 'https://www.pagina12.com.ar/suplementos/rosario12', 'https://www.pagina12.com.ar/805930-kicillof-lanzo-el-movimiento-derecho-al-futuro', 'https://www.pagina12.com.ar/buenos-aires12', 'https://www.pagina12.com.ar/805828-liga-profesional-riestra-y-talleres-quedaron-a-mano', 'https://www.facebook.com/Pagina12ok/', 'https://www.pagina12.com.ar/805572-neo-pistea-en-obras-hamlet-en-el-gta

In [ ]:
len(links)

253

In [ ]:
# Defino una función que me permita filtrar los links que corresponden a noticias de interés
def filtro_noticias( link) :
    if link is None:
        return False
    if  "https://www.pagina12.com.ar"  not in link:
        return False
    end = link.split(  "https://www.pagina12.com.ar" )[1]
    if len(end) > 1:
      return end[1].isdigit()
    else:
      return False




In [ ]:
# Aplico el filtro de noticias
noticias_filtradas = list(filter(filtro_noticias, links))

In [ ]:
len(noticias_filtradas)

99

In [ ]:
for noticia in noticias_filtradas[5:6]:
    request = requests.get(noticia)
    soup = BeautifulSoup(request.text, 'html.parser')
    print(f"Título:{soup.title}")

Título:<title>Liga Profesional: Riestra y Talleres quedaron a mano  | Empate sin goles en el Bajo Flores | Página|12</title>


In [ ]:

# Extraer títulos y epígrafes
noticias = []
for noticia in noticias_filtradas:
    request = requests.get(noticia) #Entramos al link de la notiica
    soup = BeautifulSoup(request.text, 'html.parser') #Parseamos
    headline=soup.find_all('h2')

    titulo = soup.title.text
    epigrafe = ""
    if len(headline) > 1:
      for h in headline[1::]:
        epigrafe += h.text + ". "
      noticias.append((titulo, epigrafe))



In [ ]:
len(noticias)

91

In [ ]:
noticias[0:5]

[('Clima en Buenos Aires: el pronóstico del tiempo para este sábado 22 de febrero | Sigue el calor agobiante ¿con lluvias? | Página|12',
  'El SMN estimó para esta jornada cielo mayormente nublado y probabilidad de tormentas aisladas. La mínima será de 26 grados y la máxima de 34.. '),
 ('Un legislador de LLA, contra Karina: "Tengo dudas de su transparencia" | Internas en el oficialismo por el escándalo de $LIBRA | Página|12',
  'Eugenio Casielles señaló a la Secretaria de la Presidencia y hermana de Javier Milei como responsable de la presunta cripto-estafa: "Tuvo seis reuniones con esta gente y no dijo que pasó", cuestionó el diputado porteño en diálogo con la 750.\xa0. '),
 ('Brasil: la justicia desactivó la cuenta de X de Jair Bolsonaro | Lo hizo el juez de la Corte Alexandre de Moraes, que investiga al expresidente | Página|12',
  'El juez investiga al exmandatario ultraderechista por presuntamente orquestar una trama golpista para mantenerse en el poder e impedir la investidura d

In [ ]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Definir las categorías
categorias = ["política", "economía", "deportes", "sociedad", "policiales", "tecnología", "salud", "clima", "cultura"]

# Clasificar cada noticia
for i, (titulo, epigrafe) in enumerate(noticias[:5]):
    texto_noticia = f"{titulo}. {epigrafe}"
    resultado = classifier(texto_noticia, categorias)
    print(f"Noticia {i+1}:")
    print(f"Título: {titulo}")
    print(f"Epígrafe: {epigrafe}")
    print(f"Categoría predicha: {resultado['labels'][0]}")
    print(f"Puntuación: {resultado['scores'][0]}")
    print()

Device set to use cpu


Noticia 1:
Título: Clima en Buenos Aires: el pronóstico del tiempo para este sábado 22 de febrero | Sigue el calor agobiante ¿con lluvias? | Página|12
Epígrafe: El SMN estimó para esta jornada cielo mayormente nublado y probabilidad de tormentas aisladas. La mínima será de 26 grados y la máxima de 34.. 
Categoría predicha: clima
Puntuación: 0.8321158289909363

Noticia 2:
Título: Un legislador de LLA, contra Karina: "Tengo dudas de su transparencia" | Internas en el oficialismo por el escándalo de $LIBRA | Página|12
Epígrafe: Eugenio Casielles señaló a la Secretaria de la Presidencia y hermana de Javier Milei como responsable de la presunta cripto-estafa: "Tuvo seis reuniones con esta gente y no dijo que pasó", cuestionó el diputado porteño en diálogo con la 750. . 
Categoría predicha: política
Puntuación: 0.4759426414966583

Noticia 3:
Título: Brasil: la justicia desactivó la cuenta de X de Jair Bolsonaro | Lo hizo el juez de la Corte Alexandre de Moraes, que investiga al expresidente 